In [ ]:
class EuropeanBasketCallOption:
    def __init__(self, spot_prices, sigmas, correlation_matrix, weights,
                 K, T, r, M=10000, random_seed=None):
        self.spot_prices = np.array(spot_prices)
        self.sigmas = np.array(sigmas)
        self.correlation_matrix = np.array(correlation_matrix)
        self.weights = np.array(weights)
        self.K = K
        self.T = T
        self.r = r
        self.M = M
        self.random_seed = random_seed
        self.cholesky = np.linalg.cholesky(self.correlation_matrix)
        self.basket_paths = None

    def simulate_terminal_basket(self):
        if self.random_seed is not None:
            np.random.seed(self.random_seed)

        Z = np.random.normal(size=(self.M, len(self.spot_prices)))
        correlated_Z = Z @ self.cholesky.T

        drift = (self.r - 0.5 * self.sigmas**2) * self.T
        diffusion = self.sigmas * np.sqrt(self.T)

        drift = drift[None, :]
        diffusion = diffusion[None, :]

        terminal_prices = self.spot_prices * np.exp(drift + correlated_Z * diffusion)
        basket_vals = terminal_prices @ self.weights
        self.basket_paths = basket_vals
        return basket_vals

    def price(self):
        if self.basket_paths is None:
            self.simulate_terminal_basket()
        payoffs = np.maximum(self.basket_paths - self.K, 0)
        discounted = np.exp(-self.r * self.T) * payoffs
        return np.mean(discounted)

In [ ]:
from instrument_classes import EuropeanBasketCallOption
import datetime

basket_option = EuropeanBasketCallOption(
    spot_prices=[39.72, 241.82, 21.92, 207.35],
    sigmas=[0.2106, 0.2108, 0.2676, 0.2133],
    correlation_matrix=[
        [1.00, 0.629, 0.569, 0.632],
        [0.629, 1.00, 0.080, 0.498],
        [0.569, 0.080, 1.00, 0.770],
        [0.632, 0.498, 0.770, 1.00]
    ],
    weights=[0.10, 0.35, 0.15, 0.40],
    K=175.00,
    T=(datetime.date(2025, 7, 17) - datetime.date(2025, 5, 16)).days / 365,
    r=0.041,
    M=100000,
    random_seed=42
)

option_price = basket_option.price()
print(f"European Basket Call Option Price: ${option_price:.2f}")

import numpy as np
import matplotlib.pyplot as plt

# === Inputs ===
spot_prices = np.array([39.72, 241.82, 21.92, 207.35])
volatilities = np.array([0.21065, 0.21089, 0.26761, 0.21329])
correlation_matrix = np.array([
    [1.00, 0.629, 0.569, 0.632],
    [0.629, 1.00, 0.080, 0.498],
    [0.569, 0.080, 1.00, 0.770],
    [0.632, 0.498, 0.770, 1.00]
])
weights = np.array([0.10, 0.35, 0.15, 0.40])
r = 0.041
T = 62 / 365  # ~0.1699 years
n_steps = 50
n_paths = 100
strike_price = 175

# === Setup ===
dt = T / n_steps
n_assets = len(spot_prices)
cholesky = np.linalg.cholesky(correlation_matrix)

# === Storage for basket paths ===
basket_paths = np.zeros((n_paths, n_steps + 1))
basket_paths[:, 0] = np.dot(spot_prices, weights)

# === Simulate asset and basket paths ===
current_spots = np.tile(spot_prices, (n_paths, 1))  # shape: (n_paths, n_assets)

for t in range(1, n_steps + 1):
    Z = np.random.normal(size=(n_paths, n_assets))
    correlated_Z = Z @ cholesky.T

    drift = (r - 0.5 * volatilities**2) * dt
    diffusion = volatilities * np.sqrt(dt)

    # Update asset prices for each path
    current_spots *= np.exp(drift + diffusion * correlated_Z)

    # Compute basket value per path
    basket_values = np.dot(current_spots, weights)
    basket_paths[:, t] = basket_values

# === Plotting ===
plt.figure(figsize=(12, 6))
time_grid = np.linspace(0, T, n_steps + 1)

# Color-code paths
for i in range(n_paths):
    final_value = basket_paths[i, -1]
    color = 'green' if final_value > strike_price else 'red'
    plt.plot(time_grid, basket_paths[i], lw=0.8, alpha=0.8, color=color)

# Reference line for strike
plt.axhline(y=strike_price, color='blue', linestyle='--', label="Strike Price ($175)")

plt.title("Monte Carlo Simulated Basket Paths (Green = In-the-Money)")
plt.xlabel("Time (Years)")
plt.ylabel("Basket Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
